In [6]:
import re

In [22]:
# to open and read our tweets to begin the preprocessing step
raw_text = open('tweets.csv').read()
tab_seperated = [item.split('\t') for item in raw_text.split('\n') if len(item.split('\t')) >= 4]

In [82]:
data = {}
for i in range(len(tab_seperated)):
    #extracting the filtered tweets and IDs
    data[tab_seperated[i][1]] = tab_seperated[i][4]

#Preprocessing and cleaning
data_index = data.copy()    
for tweet in data_index.keys():
    data_index[tweet] = data_index[tweet].lower()
    data_index[tweet] = re.sub('\[newline\]', ' ' , data_index[tweet])
    data_index[tweet] = re.sub('https?[^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub('[@#][^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'[0-9][^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'\w+\.[^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'[^a-zäöüß\s]', ' ', data_index[tweet])
    data_index[tweet] = re.sub(r'[^\w\s]', ' ' , data_index[tweet])

In [84]:
def cosine(text1, text2):
    import math
    import re
    def wd(t1, t2):
        wd = {}
        t1 = t1.split()
        t2 = t2.split()
        for i in t1:
            wd[i] = 0.0
        for i in t2:
            wd[i] = 0.0
        return t1, t2, wd
    
    def TF(wordDict, txt):
        tfDict = {}
        for word, count in wordDict.items():
            for i in txt:
                if word == i:
                    count +=1
            if count > 0:
                tfDict[word]=1+math.log10(count)
            else:
                tfDict[word] = 0.0
        return tfDict

    def IDF(docList):
        idfDict = {}
        N = len(docList)
        idfDict = dict.fromkeys(docList[0].keys(),0)
        for doc in docList:
            for word, val in doc.items():
                if val > 0:
                    idfDict[word] +=1
        for word, val in idfDict.items():
            idfDict[word]= math.log10(N/float(val))
        return idfDict

    def TF_IDF(tfs, idfs):
        tfidf = {}
        for word, val in tfs.items():
            tfidf[word] = val*idfs[word]
        return tfidf

    def length(tf):
        return math.sqrt(sum([(i**2) for i in list(tf.values())]))
    
    def dot(v1,v2):
        new_v = []
        for i in range(len(v1)):
            new_v.append(v1[i]*v2[i])
        return sum(new_v)
    
    txt1, txt2, wd = wd(text1, text2)
    if len(txt1) == 0 or len(txt2) == 0:
        return 0
    tf1 = TF(wd, txt1)
    tf2 = TF(wd, txt2)
    docList = [tf1,tf2]
    idf = IDF(docList)
    tf_idf1 = TF_IDF(tf1, idf)
    tf_idf2 = TF_IDF(tf2, idf)
    l1 = length(tf1)
    l2 = length(tf2)
    if l1 == 0 or l2 == 0:
        return 0
    return round(dot(list(tf1.values()), list(tf2.values()))/(l1*l2),4)

In [85]:
def top_100(tweet):
    top = []
    for id, doc in data_index.items():
        if id != tweet:
            cos = cosine(data_index[tweet], doc)
            top.append((cos,id,data[id]))
    return sorted(top)[::-1][:100]

In [86]:
data['965734992633565184']

'@knakatani @ChikonJugular @joofford @SteveBlogs11 https://t.co/WHtaRYGNSY says lifetime risk of cervical cancer in Japan is 1 in 100.  That means HPV is endemic in Japan, and screening is not working well.'

In [87]:
top = top_100('965734992633565184')
for i in top:
    print(i)

(0.4776, '982642444943605760', "@beautyloveknow1 @VABVOX Yeah, cervical cancer is so rare that the lifetime risk of getting it is 1 in 161... and cervical isn't the only kind of cancer HPV causes.[NEWLINE][NEWLINE]Please vaccinate your kids against HPV-caused cancers.  Getting the vaccine is far safer than not getting it.")
(0.4592, '991103844959506433', '@GeoffSchuler @MamaBear11011 @NeilTheMason @BillyMontana81 @EmmaGPaley @RyanMarino @LiamKav @StopVaxxedLies @MrFlu2 @what_if_007 @Luma923 @chrisvcsefalvay @Momogarden81 @Plasticdoe @ChrisJohnsonMD @DrPaolini @Vbalance03 @JakeLCrosby @doritmi @JOEWV @jkellyca @Just4TheCause @_marymason @kidoctr @SentinelM @_CWN @TinFoilAwards @jarue369 @ailsa_graham @RBuzzy1111 @ThisMomHCN @KarmicTimes @RustyPee4 @science_guy5 @itsmepanda1 @Boxmenot @FreeBeeCee @PharmaNemesis @thereal_truther @Organic_Mumzy @steffieschiltz @eTweeetz @LiamTheBrewer @HenningTveit @DUMPTRUMP7 @Dr_dad_of_3 @BlagenLogin @and_kell @kenjaques @ckjellmo And in particular, the 

In [90]:
for num, key in enumerate(data_index.keys()):
    if num == 8:
        print(num, data[key], key, data_index[key])

8 @rip_tear Tumors?[NEWLINE]Dat leg is straight mccain 965481507266727936   tumors  dat leg is straight mccain


In [89]:
top = top_100('965481507266727936')
for i in top:
    print(i)

(0.4082, '966713114770231296', '@AtypistAnne @Proud2BeAut Jeetje, dat is ernstig!')
(0.2887, '995409237324034049', '@mounira_ber تقريبا كل ال Solid tumors. المركز في ألمانيا, باد إمستال  https://t.co/ohjpgAjZkr')
(0.2887, '994992508206755840', 'The #tomato is #alive...[NEWLINE][NEWLINE]#tomate #leben #life #love #frühling #spring #liebe #plant #pflanzen #keimen #germinate https://t.co/AZx5M222Mv')
(0.2887, '992131207277240321', '#Happiness is makeable https://t.co/MXw3ZxMEkl')
(0.2887, '988902240260886528', 'Trump is obese, not large.[NEWLINE][NEWLINE]Get it straight. https://t.co/pj8omIV710')
(0.2887, '988050829562261504', '#summer is here! #nofilter #zurich #lake #bbq #love #beer #picnic #instagood #picoftheday #dusk #sunset #colors ⭐️ ⭐️ https://t.co/dtw0Ep0OrE https://t.co/ky3XrH0ARW')
(0.2887, '979869069154377730', '@poppy is hip')
(0.2887, '975546597172969477', '#gratitude is the #best #medicine #meditation #nature #hiking #love #joy #❤️ #newearth… https://t.co/X9QnEYEmvI')
(0.28

In [91]:
for num, key in enumerate(data_index.keys()):
    if num == 22:
        print(num, data[key], key, data_index[key])

22 Und: Syphilis kommt wieder.[NEWLINE]Ja, eine P(R)EP/Therapie kann das HIV Übertragungs-/Infektionsrisiko auf (nahezu) null senken.[NEWLINE]Aber Syphilis, der kleine Ficker, ist schon längst wieder da.[NEWLINE]Syphilis tut am Anfang nicht weh. Aber am Ende beißt sie einem doch in den Arsch... 965711648278437888 und  syphilis kommt wieder  ja  eine p r ep therapie kann das hiv übertragungs  infektionsrisiko auf  nahezu  null senken  aber syphilis  der kleine ficker  ist schon längst wieder da  syphilis tut am anfang nicht weh  aber am ende beißt sie einem doch in den  


In [92]:
top = top_100('965711648278437888')
for i in top:
    print(i)

(0.3482, '982040536419840000', 'Ich habe gerade erfahren, dass heute ein Tag früher ist, als ich dachte! Es ist erst Donnerstag auf Freitag, nicht Freitag auf Samstag.[NEWLINE]Und da ich meinte, dass das letzte Kapitel Samstag kommt, es aber ja doch Fr. kommt hab ich den Tag anscheinend schon Anfang der Woche verloren')
(0.3479, '959206122036088832', 'Da kommt diese schiere Wut schon wieder in mir hoch. Die Frau als das schwache Geschlecht, aber die ganzen Nebenwirkungen und Risiken kann sie ja tragen, weil der Mann sich dann nicht um die Verhütung kümmern muss. https://t.co/ntvSBl86Ye')
(0.3305, '970810318984228865', '@TohsakaShirou XD Ja gut, der hat halt seinen eigenen Kopf..  Aber ein wenig Einfluss kann man da schon drauf habe, ist doch besser als gar nichts :p[NEWLINE]Und ich versuch jetzt wieder zu schlafen! Der Alarm in der Nachbarschaft hält endlich wieder sein Maul und ich hoffe, das bleibt auch so ;-;')
(0.3278, '992450802198417413', '@mastermikeg @FreeBourgeoisie @FrankBreu